In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns

In [4]:
# Load the new_data set
dr = pd.read_csv('/Users/frieda/Desktop/new_data.csv')

In [5]:
dr

,Unnamed: 0,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2
...,...,...,...,...,...,...,...,...,...,...
9999995,9999995,2019-11-07 05:08:21 UTC,view,1005115,2053013555631882655,electronics.smartphone,apple,915.69,517800555,c6b764e0-561c-4c17-a546-2e922196bb49
9999996,9999996,2019-11-07 05:08:21 UTC,view,12707281,2053013553559896355,NaN,gislaved,90.86,539095445,b1673b33-d0ff-4a09-bf57-56dedc3d016e
9999997,9999997,2019-11-07 05:08:21 UTC,view,3601405,2053013563810775923,appliances.kitchen.washer,beko,180.16,548916637,e020a307-551c-4e58-a787-126701d30d41
9999998,9999998,2019-11-07 05:08:21 UTC,view,12711053,2053013553559896355,NaN,tunga,32.69,521586999,c8c48941-2ca1-4ce4-8d44-c96eb5df4fb5


In [6]:
dr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Unnamed: 0     int64  
 1   event_time     object 
 2   event_type     object 
 3   product_id     int64  
 4   category_id    int64  
 5   category_code  object 
 6   brand          object 
 7   price          float64
 8   user_id        int64  
 9   user_session   object 
dtypes: float64(1), int64(4), object(5)
memory usage: 762.9+ MB


In [7]:
dr.describe()

,Unnamed: 0,product_id,category_id,price,user_id
count,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07
mean,5.000000e+06,1.100008e+07,2.057828e+18,2.925741e+02,5.354292e+08
std,2.886751e+06,1.228346e+07,1.943274e+16,3.538111e+02,2.037214e+07
min,0.000000e+00,1.000365e+06,2.053014e+18,0.000000e+00,1.030022e+07
25%,2.500000e+06,1.005252e+06,2.053014e+18,6.667000e+01,5.159950e+08
50%,5.000000e+06,5.100722e+06,2.053014e+18,1.653100e+02,5.306267e+08
75%,7.499999e+06,1.640001e+07,2.053014e+18,3.654600e+02,5.548916e+08
max,9.999999e+06,6.210003e+07,2.180737e+18,2.574070e+03,5.684226e+08


In [21]:
dr2 = dr.dropna(how = 'any', subset = ['category_code','brand','price','user_id'])
print(dr2)

         Unnamed: 0               event_time event_type  product_id  \
0                 0  2019-11-01 00:00:00 UTC       view     1003461   
1                 1  2019-11-01 00:00:00 UTC       view     5000088   
3                 3  2019-11-01 00:00:01 UTC       view     3601530   
4                 4  2019-11-01 00:00:01 UTC       view     1004775   
5                 5  2019-11-01 00:00:01 UTC       view     1306894   
...             ...                      ...        ...         ...   
9999991     9999991  2019-11-07 05:08:21 UTC       view     1005177   
9999994     9999994  2019-11-07 05:08:21 UTC       view     4804295   
9999995     9999995  2019-11-07 05:08:21 UTC       view     1005115   
9999997     9999997  2019-11-07 05:08:21 UTC       view     3601405   
9999999     9999999  2019-11-07 05:08:21 UTC       cart     1005121   

                 category_id                category_code    brand   price  \
0        2053013555631882655       electronics.smartphone   xiaomi  4

Let's do a t-test first, to see if there is a significant difference between the price between two groups: apple and samsung:

In [22]:
samsung = dr2[dr2['brand'] == 'samsung']['price']
apple = dr2[dr2['brand'] == 'apple']['price']

t_stat, p_val = stats.ttest_ind(samsung, apple)
#an independent two-sample t-test, which is a statistical test 
# to determine if there is a significant difference between the means 
# of two independent groups.

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_val}")

T-statistic: -925.8345012624801
P-value: 0.0


null hypythosis: there is not significant difference between the means of price of samsung and apple
because the p-value is too small, we reject the null hypothesis. 

In [28]:

dr3 = dr2.loc[dr2['brand'].isin(['samsung', 'apple']), ['brand', 'price']]
#.loc is a method in pandas used for label-based indexing or 
# selecting data from a DataFrame. It allows you to access a subset
#  of rows and columns based on labels or a boolean mask.\
dr3

,brand,price
9,apple,732.07
14,samsung,411.83
27,apple,949.47
39,apple,458.28
40,samsung,503.09
...,...,...
9999989,apple,473.37
9999990,samsung,306.05
9999991,samsung,900.64
9999995,apple,915.69


Next, let's compute the Chi-squared test:

In [29]:

# Convert 'sepal_width' to a categorical variable
dr3['price_table'] = pd.cut(dr3['price'], bins=[0, 100, 300, 500,1000])

# Create a contingency table
contingency_table = pd.crosstab(dr3['brand'], dr3['price_table'])

chi2, p_val, dof, expected = stats.chi2_contingency(contingency_table)
# determine whether there is a significant association between two 
# categorical variables in a dataset.
print(f"Chi-squared: {chi2}")
print(f"P-value: {p_val}")
#You can use the outputs of stats.chi2_contingency to assess the statistical 
# significance of the association between the variables. 
# If the p-value is below a chosen significance level (e.g., 0.05),
#  you can reject the null hypothesis of independence and conclude 
# that there is evidence of an association between the variables.

Chi-squared: 495202.9687975902
P-value: 0.0


In [37]:
dr3

,brand,price,price_table
9,apple,732.07,"(500, 1000]"
14,samsung,411.83,"(300, 500]"
27,apple,949.47,"(500, 1000]"
39,apple,458.28,"(300, 500]"
40,samsung,503.09,"(500, 1000]"
...,...,...,...
9999989,apple,473.37,"(300, 500]"
9999990,samsung,306.05,"(300, 500]"
9999991,samsung,900.64,"(500, 1000]"
9999995,apple,915.69,"(500, 1000]"


In [30]:
print(contingency_table)

price_table  (0, 100]  (100, 300]  (300, 500]  (500, 1000]
brand                                                     
apple            5630       96580      162916       453880
samsung         56165      630692      212538       188421


We do see a statistical significance. Let's move on to an ANOVA test, to see if there is a difference between the means of all 3 groups

In [31]:
dr4 = dr3[dr3['brand'] == 'samsung']['price']
dr5 = dr3[dr3['brand'] == 'apple']['price']
dr6 = dr2[dr2['brand'] == 'xiaomi']['price']

f_stat, p_val = stats.f_oneway(dr4, dr5, dr6)
#(ANOVA) test to determine whether there are any significant 
# differences among the means of two or more groups or populations.

print(f"F-statistic: {f_stat}")
print(f"P-value: {p_val}")

F-statistic: 766632.6927309786
P-value: 0.0


This indicates strong evidence of at least one significant difference among the group means.

Now let's calculate Pearson's correlation between two continuous variables:

In [33]:
pearson_corr, p_val = stats.pearsonr(dr3[dr3['brand'] == 'samsung']['price'], dr3[dr3['brand'] == 'apple']['price'])
print(f"Pearson's correlation: {pearson_corr}")
print(f"P-value: {p_val}")

ValueError: x and y must have the same length.

In [36]:
revenue_apple = dr3[dr3['brand'] == 'apple']['price'].sum()
revenue_apple

793567940.5999998